In [1]:
#Background situation

In [9]:
#Import Libraries
import pandas as pd
from pathlib import Path
import os
import hvplot.pandas
import numpy as np
from datetime import date
import seaborn as sns
import json
import kaggle

In [10]:
#Import file from API (MUST HAVE KAGGLE INSTALLED: ("conda install kaggle")
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()


api.dataset_download_file('syuzai/perth-house-prices','all_perth_310121.csv')

False

In [11]:
#CSV file was unzipped from downloaded file

#Read CSV file
all_perth_data = pd.read_csv("all_perth_310121.csv")

In [12]:
#Drop NAs
all_perth_data = all_perth_data.dropna()

#Confirming all nulls have been removed
all_perth_data.isnull().sum()

ADDRESS             0
SUBURB              0
PRICE               0
BEDROOMS            0
BATHROOMS           0
GARAGE              0
LAND_AREA           0
FLOOR_AREA          0
BUILD_YEAR          0
CBD_DIST            0
NEAREST_STN         0
NEAREST_STN_DIST    0
DATE_SOLD           0
POSTCODE            0
LATITUDE            0
LONGITUDE           0
NEAREST_SCH         0
NEAREST_SCH_DIST    0
NEAREST_SCH_RANK    0
dtype: int64

In [13]:
#Remove \r from the Date_Sold column
all_perth_data['DATE_SOLD'] = all_perth_data['DATE_SOLD'].str.strip('\r')
all_perth_data['DATE_SOLD'] = pd.to_datetime(all_perth_data['DATE_SOLD'])

In [14]:
# Second option concatenation with another csv file
project_path_2 = Path("./residential_property_price.csv")

In [15]:
# Reading the residential_property_price data 
price_df = pd.read_csv(project_path_2)
#Filtering data points where MEDIAN HOUSE PRICE is greater than zero. 
price_df = price_df[price_df['MEDIAN HOUSE PRICE'] > 0]
#Setting the index of the data frame as SUBURB
price_df = price_df.set_index(price_df["SUBURB"])
#Displaying the first few rows of the Data frame
price_df.head()

,SUBURB,MEDIAN HOUSE PRICE,GROWTH PERCENTAGE
SUBURB,,,
Alexander Heights,Alexander Heights,490000,8.90%
Alfred Cove,Alfred Cove,921000,17.00%
Alkimos,Alkimos,452500,10.40%
Applecross,Applecross,1735000,-0.60%
Ardross,Ardross,1165000,15.90%


In [16]:
#Selecting specific columns from the data frame
suburbs_df = all_perth_data[['SUBURB', 'LATITUDE', 'LONGITUDE']].groupby(['SUBURB']).mean()
#Concatinating both the data frames.
suburbs_df_price = pd.concat([price_df, suburbs_df], axis="columns", join="inner")
#Displaying first few data points of data frame
suburbs_df_price.head()

,SUBURB,MEDIAN HOUSE PRICE,GROWTH PERCENTAGE,LATITUDE,LONGITUDE
SUBURB,,,,,
Alexander Heights,Alexander Heights,490000,8.90%,-31.830310,115.866293
Alfred Cove,Alfred Cove,921000,17.00%,-32.033908,115.816117
Alkimos,Alkimos,452500,10.40%,-31.618988,115.691004
Applecross,Applecross,1735000,-0.60%,-32.013569,115.838399
Ardross,Ardross,1165000,15.90%,-32.026691,115.838054


In [17]:
#Turning Growth Percentage to Float data type to sort correctly
suburbs_df_price['GROWTH PERCENTAGE'] = suburbs_df_price['GROWTH PERCENTAGE'].str.strip('%')
suburbs_df_price['GROWTH PERCENTAGE'] = suburbs_df_price['GROWTH PERCENTAGE'].astype(float)

In [18]:
#Full Geo map grouped by suburbs
suburbs_df_price.hvplot.points('LONGITUDE',
                                       'LATITUDE', 
                                       geo=True,
                                       color='MEDIAN HOUSE PRICE',
                                       size='MEDIAN HOUSE PRICE',
                                       scale=0.013,
                                       tiles='OSM',
                                       frame_width=700,
                                       frame_height=500,
                                       hover = True,
                                       hover_cols=['SUBURB','MEDIAN HOUSE PRICE']
                                        )

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [LONGITUDE,LATITUDE]   (MEDIAN HOUSE PRICE,SUBURB)

#CLIENT 1

Client 1 has come to us asking for a two bedroom house within their $300,000. They're not bothered about the land size, as long as it is within 2km of the CBD

In [31]:
#Input of client requirements
#Creating a new data frame df by dropping Build_year, postcode, date sold
df = all_perth_data.drop(columns=['BUILD_YEAR', 'POSTCODE', 'DATE_SOLD'])
#Requesting the clients property requirements
plan_budget = int(input('Please enter your budget : '))
bedrooms = int(input('Please enter the number of Bedrooms : '))
landsize = int(input('Please enter your desired minimum land size : '))
cbddist = int(input('Please enter the maximum distance from CBD in metres: '))
#Filtering out properties based on clients requirements
client1 = df.loc[(all_perth_data['BEDROOMS'] >= bedrooms)
                               & (df['PRICE'] <= plan_budget)
                               & (df['CBD_DIST'] <= cbddist)
                               & (df['LAND_AREA'] >= landsize)]
#Displaying the properties which meet clients requirements
client1

Please enter your budget :  300000
Please enter the number of Bedrooms :  2
Please enter your desired minimum land size :  1
Please enter the maximum distance from CBD in metres:  2000


,ADDRESS,SUBURB,PRICE,BEDROOMS,BATHROOMS,GARAGE,LAND_AREA,FLOOR_AREA,CBD_DIST,NEAREST_STN,NEAREST_STN_DIST,LATITUDE,LONGITUDE,NEAREST_SCH,NEAREST_SCH_DIST,NEAREST_SCH_RANK
20398,39/193 Hay Street,East Perth,258000,2,1,1.0,2425,80,1600,Claisebrook Station,1100,-31.959205,115.873203,MERCEDES COLLEGE,0.665447,15.0
23351,47 Smith Street,Highgate,284000,2,1,2.0,238,101,1700,East Perth Station,644,-31.941076,115.870885,PERTH COLLEGE,1.183227,9.0
28248,65/273 Hay Street,East Perth,290000,2,1,1.0,1882,80,1300,McIver Station,822,-31.958307,115.870264,MERCEDES COLLEGE,0.378614,15.0


In [21]:
#Visualization displaying the properties which meet clients requirements
client1.hvplot.points('LONGITUDE',
                                       'LATITUDE', 
                                       geo=True,
                                       color='ADDRESS',
                                       tiles='OSM',
                                       size= 'PRICE',
                                       scale= 0.05,
                                       frame_width=700,
                                       frame_height=500,
                                       hover = True,
                                       hover_cols=['SUBURB','PRICE', 'CBD_DIST', 'BEDROOMS']
                                        )

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [LONGITUDE,LATITUDE]   (ADDRESS,PRICE,SUBURB,CBD_DIST,BEDROOMS)

In [32]:
#Creating an empty list
temp_list = {}
#Loading the suburb which the property is located in.
temp_list = client1['SUBURB']
#Printing the median price and the growth rate of the property.
display(price_df[price_df['SUBURB'].isin(temp_list)])

,SUBURB,MEDIAN HOUSE PRICE,GROWTH PERCENTAGE
SUBURB,,,
East Perth,East Perth,1950000,77.30%
Highgate,Highgate,994000,10.50%


In [23]:
#Defining mortgage calculator 
def mortgagecalculator (mortgage, years, interest):
#Converting decimal input to floating point value
    interest = interest / 100
#Calculating the number of periods per year   
    nper = years*12
#Calculating the monthly intrest payment
    interest_monthly = interest/12
    numerator = interest_monthly*((1+interest_monthly)**nper)
    denominator = (1+interest_monthly)**nper-1
#Returning the result to function call and formatting the displayes numerical.
    payment = float("{0:.2f}".format(mortgage*numerator/denominator))
    return(payment)

In [34]:
#Collecting clients mortgage details.
#Requesting client the loan amount
mortgate_amount=int(input('What is the mortgage amount? '))
#Enquiring clint of the duration to repay the loan
no_of_years=int(input('For how many years? '))
#Collecting the intrest rate information from the bank
intrest_rate=int(input('What interest rate? '))
#Calling the function to provide the monthly repayment
mortgagecalculator(mortgate_amount,no_of_years,intrest_rate)

What is the mortgage amount?  280000
For how many years?  30
What interest rate?  4


1336.76

In [25]:
#Definition of stamp duty calculator
#Calculating the stamp duty based on the value of the property.
def stampdutycalculator (purchase_price):
    if purchase_price <= 120000:
        stamp_duty = purchase_price*0.019
    elif purchase_price > 120000 and purchase_price <= 150000:
        stamp_duty = 2280 + (purchase_price - 120000) * 0.0285
    elif purchase_price > 150000 and purchase_price <= 360000:
        stamp_duty = 3135 + (purchase_price - 150000) * 0.038
    elif purchase_price > 360000 and purchase_price <= 725000:
        stamp_duty = 11175 + (purchase_price - 360000) * 0.0475
    else:
        stamp_duty = 28453 + (purchase_price - 725000) * 0.0515
#Returning the calculated stamp duty to the function call   
    return stamp_duty

In [35]:
#Entering the property value dynamically.
property_value=int(input('What is the property value? '))
#Calling the function definition to calculate the property_value
stampdutycalculator(property_value)

What is the property value?  285000


8265.0

In [37]:
deposit = (property_value * 0.2) + 8265
deposit

65265.0

Client 2 has requested a larger house and property that can be further away from the CBD if needed. They also have a larger budget to facilitate this request.

In [38]:
#Input of client 2's requirements
#Requesting the clients property requirements
plan_budget2 = int(input('Please enter your budget : '))
bedrooms2 = int(input('Please enter the number of Bedrooms : '))
landsize2 = int(input('Please enter your desired minimum land size : '))
cbddist2 = int(input('Please enter the maximum distance from CBD in metres: '))
#Filtering out properties based on clients requirements 
client2 = df.loc[(all_perth_data['BEDROOMS'] >= bedrooms2)
                               & (df['PRICE'] <= plan_budget2)
                               & (df['CBD_DIST'] <= cbddist2)
                               & (df['LAND_AREA'] >= landsize2)]
#Displaying the properties which meet clients requirements
client2

Please enter your budget :  900000
Please enter the number of Bedrooms :  5
Please enter your desired minimum land size :  800
Please enter the maximum distance from CBD in metres:  7000


,ADDRESS,SUBURB,PRICE,BEDROOMS,BATHROOMS,GARAGE,LAND_AREA,FLOOR_AREA,CBD_DIST,NEAREST_STN,NEAREST_STN_DIST,LATITUDE,LONGITUDE,NEAREST_SCH,NEAREST_SCH_DIST,NEAREST_SCH_RANK
10254,2 Crawshaw Crescent,Manning,850000,5,2,2.0,856,105,6100,Canning Bridge Station,884,-32.006520,115.86473,COMO SECONDARY COLLEGE,0.684223,87.0
11060,20 Downey Drive,Manning,769000,5,2,3.0,800,158,6700,Canning Bridge Station,1000,-32.012480,115.86644,COMO SECONDARY COLLEGE,1.076696,87.0
12981,23 Goss Avenue,Manning,825000,5,2,2.0,870,184,6400,Canning Bridge Station,1600,-32.008656,115.87305,COMO SECONDARY COLLEGE,0.583289,87.0
15009,27 Ferrar Street,Menora,875000,5,2,1.0,835,200,3400,Mount Lawley Station,2200,-31.922160,115.86349,MOUNT LAWLEY SENIOR HIGH SCHOOL,0.600062,20.0
23095,46 Grosvenor Road,Bayswater,699000,5,1,2.0,859,157,5100,Meltham Station,543,-31.926580,115.90329,CHISHOLM CATHOLIC COLLEGE,1.695584,29.0
23590,49 Acton Avenue,Rivervale,750000,5,2,5.0,906,228,5400,Burswood Station,1600,-31.956110,115.91609,BELMONT CITY COLLEGE,1.758623,57.0
27574,60 Inverness Crescent,Menora,775000,5,3,4.0,822,55,3700,Perth Station,3600,-31.918580,115.85830,MOUNT LAWLEY SENIOR HIGH SCHOOL,1.120187,20.0


In [39]:
#New geo plot that contains house options for client B
client2.hvplot.points('LONGITUDE',
                                       'LATITUDE', 
                                       geo=True,
                                       color='ADDRESS',
                                       tiles='OSM',
                                       size= 'PRICE',
                                       scale= 0.05,
                                       frame_width=700,
                                       frame_height=500,
                                       hover = True,
                                       hover_cols=['SUBURB','PRICE', 'CBD_DIST', 'BEDROOMS']
                                        )

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [LONGITUDE,LATITUDE]   (ADDRESS,PRICE,SUBURB,CBD_DIST,BEDROOMS)

In [41]:
#Creating an empty list
temp_list2 = {}
#Loading the suburb which the property is located in.
temp_list2 = client2['SUBURB']
#Printing the median price and the growth rate of the property.
display(price_df[price_df['SUBURB'].isin(temp_list2)])

,SUBURB,MEDIAN HOUSE PRICE,GROWTH PERCENTAGE
SUBURB,,,
Bayswater,Bayswater,676500,9.10%
Manning,Manning,870000,17.60%
Menora,Menora,1300000,12.10%
Rivervale,Rivervale,617500,12.30%


In [44]:
#Collecting clients mortgage details.
#Requesting client the loan amount
mortgate_amount=int(input('What is the mortgage amount? '))
#Enquiring clint of the duration to repay the loan
no_of_years=int(input('For how many years? '))
#Collecting the intrest rate information from the bank
intrest_rate=int(input('What interest rate? '))
#Calling the function to provide the monthly repayment
mortgagecalculator(mortgate_amount,no_of_years,intrest_rate)

What is the mortgage amount?  700000
For how many years?  30
What interest rate?  4


3341.91

In [43]:
#Entering the property value dynamically.
property_value=int(input('What is the property value? '))
#Calling the function definition to calculate the property_value
stampdutycalculator(property_value)

What is the property value?  769000


30719.0

In [14]:
#A couple of analysis questions including: If you had unlimited money, where would be the best value for money for an investment price per land area etc

#What are the top 3 suburbs to invest in by future growth percentage

#What are the top 3 suburbs to avoid by future growth percentage

In [115]:
suburbs_df_price = suburbs_df_price.sort_values(['GROWTH PERCENTAGE'], ascending=False)

In [117]:
display(suburbs_df_price.head(5), suburbs_df_price.tail(5))

,SUBURB,MEDIAN HOUSE PRICE,GROWTH PERCENTAGE,LATITUDE,LONGITUDE
SUBURB,,,,,
Gnangara,Gnangara,770000,94.6,-31.775330,115.848851
Hovea,Hovea,940000,82.5,-31.887604,116.113356
East Perth,East Perth,1950000,77.3,-31.952803,115.874626
Upper Swan,Upper Swan,790000,61.2,-31.769674,116.029006
Watermans Bay,Watermans Bay,1670000,51.8,-31.849449,115.755512


,SUBURB,MEDIAN HOUSE PRICE,GROWTH PERCENTAGE,LATITUDE,LONGITUDE
SUBURB,,,,,
Maylands,Maylands,655000,-5.1,-31.934201,115.900674
Sawyers Valley,Sawyers Valley,565000,-5.8,-31.893090,116.220120
Herne Hill,Herne Hill,370000,-11.9,-31.829031,116.027816
Darling Downs,Darling Downs,530000,-13.3,-32.199243,115.992782
Carmel,Carmel,555000,-14.6,-32.014680,116.072370


In [45]:
#Future development